In [5]:
import pandas as pd
import numpy as np
import sys

sys.path.append('/home/yuliya/repos/cosybio/FedProt/evaluation_utils/')
from evaluation import evaluation_func as fp_eval

from collections import OrderedDict

# Config

In [6]:
root_dir = "/home/yuliya/repos/cosybio/FedProt/evaluation/simulated"

fedprot_name="DPE.tsv"
deqms_name="res.tsv"
fisher_name="MA_CM.tsv"
rem_name="MA_REM.tsv"
stouffer_name="MA_Stouffer.tsv"
rankprod_name="MA_RankProd.tsv"

number_of_runs = 51

lfc_thr = 1
pval_thr = 0.05


# balanced - balanced -- CHANGE!
# fedprot_name="DPE.tsv"

In [7]:
def read_data(workdir, prefix):
    df = fp_eval.read_results(
                workdir, 
                fedprot_name=f"{prefix}{fedprot_name}",
                deqms_name=f"{prefix}{deqms_name}",
                fisher_name=f"{prefix}{fisher_name}",
                rem_name=f"{prefix}{rem_name}",
                stouffer_name=f"{prefix}{stouffer_name}",
                rankprod_name=f"{prefix}{rankprod_name}"
    )
    return df

def log_transform_df(df):
    df = df.replace(0,1e-300)
    df["pv_DEqMS"] = -np.log10(df["pv_DEqMS"])
    for m in ["FedProt", "Fisher","Stouffer","REM","RankProd"]:
        df["pv_"+m] = -np.log10(df["pv_"+m])
    return df


In [8]:
for mode in ["balanced", "imbalanced", "mild_imbalanced"]: #, "imbalanced_balanced", "balanced_imbalanced"

    summarized_stats_F1 = {}
    summarized_stats_differences = {}
    summarized_stats_cor_error = {}
    # same for logFC
    summarized_stats_differences_LF = {}
    summarized_stats_cor_error_LF = {}

    FP_df = pd.DataFrame(columns=["FedProt", "Fisher", "Stouffer", "REM", "RankProd"])
    FN_df = pd.DataFrame(columns=["FedProt", "Fisher", "Stouffer", "REM", "RankProd"])
    Jaccard_df = pd.DataFrame(columns=["FedProt", "Fisher", "Stouffer", "REM", "RankProd"])

    mindiff_df = pd.DataFrame(columns=["FedProt", "Fisher", "Stouffer", "REM", "RankProd"])
    maxdiff_df = pd.DataFrame(columns=["FedProt", "Fisher", "Stouffer", "REM", "RankProd"])
    meandiff_df = pd.DataFrame(columns=["FedProt", "Fisher", "Stouffer", "REM", "RankProd"])

    r_df = pd.DataFrame(columns=["FedProt", "Fisher", "Stouffer", "REM", "RankProd"])
    rho_df = pd.DataFrame(columns=["FedProt", "Fisher", "Stouffer", "REM", "RankProd"])

    mindiffLF_df = pd.DataFrame(columns=["FedProt", "Fisher", "Stouffer", "REM", "RankProd"])
    maxdiffLF_df = pd.DataFrame(columns=["FedProt", "Fisher", "Stouffer", "REM", "RankProd"])
    meandiffLF_df = pd.DataFrame(columns=["FedProt", "Fisher", "Stouffer", "REM", "RankProd"])

    rLF_df = pd.DataFrame(columns=["FedProt", "Fisher", "Stouffer", "REM", "RankProd"])
    rhoLF_df = pd.DataFrame(columns=["FedProt", "Fisher", "Stouffer", "REM", "RankProd"])

    jaccard_topN_df = pd.DataFrame()

    # create a dictionary of dataframes for each run
    for i in range(1, number_of_runs):
        workdir = root_dir + f"/{mode}/results"
        prefix = f"/{i}_"
        df = read_data(workdir, prefix)
        log_df = log_transform_df(df)

        # create dfs with basic stats
        pval_stats = fp_eval.calc_stats(
            log_df, 
            lfc_thr=lfc_thr, adj_pval_thr=pval_thr,
            stats=["FP", "FN", "Jaccard",
                   "MinDiff", "MeanDiff", "MaxDiff", 
                   "r", "ρ"],
            methods=["FedProt", "Fisher", "Stouffer", "REM", "RankProd"],
            column_name="pv_",
            top_genes=-1
        )

        FP_df.loc[i] = pval_stats["FP"].T
        FN_df.loc[i] = pval_stats["FN"].T
        Jaccard_df.loc[i] = pval_stats["Jaccard"].T
        mindiff_df.loc[i] = pval_stats["MinDiff"].T
        meandiff_df.loc[i] = pval_stats["MeanDiff"].T
        maxdiff_df.loc[i] = pval_stats["MaxDiff"].T
        r_df.loc[i] = pval_stats["r"].T
        rho_df.loc[i] = pval_stats["ρ"].T

        logFC_stats = fp_eval.calc_stats(
            log_df, 
            lfc_thr=lfc_thr, adj_pval_thr=pval_thr,
            stats=["MinDiff", "MeanDiff", "MaxDiff", 
                   "r", "ρ"],
            methods=["FedProt", "Fisher", "Stouffer", "REM", "RankProd"],
            column_name="lfc_",
            top_genes=-1
        )

        mindiffLF_df.loc[i] = logFC_stats["MinDiff"].T
        meandiffLF_df.loc[i] = logFC_stats["MeanDiff"].T
        maxdiffLF_df.loc[i] = logFC_stats["MaxDiff"].T
        rLF_df.loc[i] = logFC_stats["r"].T
        rhoLF_df.loc[i] = logFC_stats["ρ"].T

        # top N jaccard
        top_n_genes = np.arange(5, 700, 5)
        stats = {}
        for j in range(len(top_n_genes)):  
            confusion_matrix = fp_eval.calc_stats_TOP(
                log_df, 
                stats=["Jaccard"],
                top_genes=top_n_genes[j])
            stats[top_n_genes[j]] = confusion_matrix["Jaccard"]
        stats = pd.DataFrame.from_dict(stats).rename_axis("Method").reset_index()

        jaccard_topN_df = pd.concat([jaccard_topN_df, stats], ignore_index=False)

    # save dfs
    workdir = root_dir + f"/{mode}/aggreagated"
    FP_df.to_csv(f"{workdir}/FP.csv", index=True)
    FN_df.to_csv(f"{workdir}/FN.csv", index=True)
    Jaccard_df.to_csv(f"{workdir}/F1.csv", index=True)
    mindiff_df.to_csv(f"{workdir}/mindiff.csv", index=True)
    meandiff_df.to_csv(f"{workdir}/meandiff.csv", index=True)
    maxdiff_df.to_csv(f"{workdir}/maxdiff.csv", index=True)
    r_df.to_csv(f"{workdir}/r.csv", index=True)
    rho_df.to_csv(f"{workdir}/rho.csv", index=True)

    mindiffLF_df.to_csv(f"{workdir}/mindiff_LF.csv", index=True)
    meandiffLF_df.to_csv(f"{workdir}/meandiff_LF.csv", index=True)
    maxdiffLF_df.to_csv(f"{workdir}/maxdiff_LF.csv", index=True)
    rLF_df.to_csv(f"{workdir}/r_LF.csv", index=True)
    rhoLF_df.to_csv(f"{workdir}/rho_LF.csv", index=True)

    jaccard_topN_df.to_csv(f"{workdir}/jaccard_topN.csv", index=True)

    # Create summary stats for p-values (using df.aggregate(["mean", "std"]) and adding two values for each method)
    summarized_stats_F1["FP"] = FP_df.aggregate(["mean", "std"]).T
    summarized_stats_F1["FN"] = FN_df.aggregate(["mean", "std"]).T
    summarized_stats_F1["Jaccard"] = Jaccard_df.aggregate(["mean", "std"]).T

    summarized_stats_differences["MinDiff"] = mindiff_df.aggregate(["mean", "std"]).T
    summarized_stats_differences["MeanDiff"] = meandiff_df.aggregate(["mean", "std"]).T
    summarized_stats_differences["MaxDiff"] = maxdiff_df.aggregate(["mean", "std"]).T

    summarized_stats_cor_error["r"] = r_df.aggregate(["mean", "std"]).T
    summarized_stats_cor_error["rho"] = rho_df.aggregate(["mean", "std"]).T

    # Create summary stats for logFC (using df.aggregate(["mean", "std"]) and adding two values for each method)
    summarized_stats_differences_LF["MinDiff"] = mindiffLF_df.aggregate(["mean", "std"]).T
    summarized_stats_differences_LF["MeanDiff"] = meandiffLF_df.aggregate(["mean", "std"]).T
    summarized_stats_differences_LF["MaxDiff"] = maxdiffLF_df.aggregate(["mean", "std"]).T

    summarized_stats_cor_error_LF["r"] = rLF_df.aggregate(["mean", "std"]).T
    summarized_stats_cor_error_LF["rho"] = rhoLF_df.aggregate(["mean", "std"]).T

    # save summary stats as df with two level index
    summarized_stats_F1 = pd.concat(summarized_stats_F1, axis=1)
    summarized_stats_differences = pd.concat(summarized_stats_differences, axis=1)
    summarized_stats_cor_error = pd.concat(summarized_stats_cor_error, axis=1)

    summarized_stats_differences_LF = pd.concat(summarized_stats_differences_LF, axis=1)
    summarized_stats_cor_error_LF = pd.concat(summarized_stats_cor_error_LF, axis=1)

    # save summary stats to csv, but save in scientific notation
    summarized_stats_F1.to_csv(f"{workdir}/summarized_stats_F1.csv", index=True, float_format="%.2E")
    summarized_stats_differences.to_csv(f"{workdir}/summarized_stats_differences.csv", index=True, float_format="%.2E")
    summarized_stats_cor_error.to_csv(f"{workdir}/summarized_stats_cor_error.csv", index=True, float_format="%.2E")
    summarized_stats_differences_LF.to_csv(f"{workdir}/summarized_stats_differences_LF.csv", index=True, float_format="%.2E")
    summarized_stats_cor_error_LF.to_csv(f"{workdir}/summarized_stats_cor_error_LF.csv", index=True, float_format="%.2E")


Results loaded for DEqMS with 6000 proteins.


Results loaded for FedProt with 6000 proteins.
Results loaded for Fisher with 6000 proteins.
Results loaded for REM with 6000 proteins.
Results loaded for Stouffer with 6000 proteins.
Results loaded for RankProd with 6000 proteins.
Results loaded from /home/yuliya/repos/cosybio/FedProt/evaluation/simulated/balanced/results with 6000 genes. Adj.p-values were not log-transformed.
Performance metrics calculated for all genes.
Calculating corrs. Using p-vals - log-transformed.
Correlations computed for all genes from pv_ columns.
Correlations computed for all genes from lfc_ columns.
Results loaded for DEqMS with 6000 proteins.
Results loaded for FedProt with 6000 proteins.
Results loaded for Fisher with 6000 proteins.
Results loaded for REM with 6000 proteins.
Results loaded for Stouffer with 6000 proteins.
Results loaded for RankProd with 6000 proteins.
Results loaded from /home/yuliya/repos/cosybio/FedProt/evaluation/simulated/balanced/results with 6000 genes. Adj.p-values were not log-